In [ ]:
import re
import glob
import pathlib
from IPython.display import display, display_markdown

In [ ]:
outputfiles = glob.glob("../check/results/*.out")
display(outputfiles[:10])

outputfile = outputfiles[0]
display(outputfile)

In [ ]:
for outputfile in outputfiles:
    print(f"Running {outputfile}")
    # Read contents
    with open(outputfile, "r") as f:
        contents = f.read()
        
    permutations = re.findall(r"^@REORDERING\:?\s+(?:(START)|(\S+) <--> (\S+))$", contents, flags=re.MULTILINE)

    # Find last occurrence of "@REORDERING START", and take the permutation from there.
    permutations = permutations[-permutations[::-1].index(("START", "", "")):]
    permutations = [(j, k) for (i, j, k) in permutations]
    permutations = [(str.replace(j, "t_x", "x"), str.replace(k, "t_x", "x")) for j, k in permutations]

    constraints = re.findall(r"^@CONSTRAINT (.*)$", contents, flags=re.MULTILINE)

    origcipfile = "../instances/flowersnark/" + pathlib.Path(outputfile).name[:-4]

    cipfile_withctrs = f"{origcipfile[:-4]}_withctrs.cip"
    cipfile_relabelonly = f"{origcipfile[:-4]}_relabel.cip"
    cipfile_withrelabel = f"{origcipfile[:-4]}_withctrs_relabel.cip"

    with open(origcipfile, "r") as f:
        lines = f.readlines()

    sttline = lines.index("STATISTICS\n")
    objline = lines.index("OBJECTIVE\n")
    varline = lines.index("VARIABLES\n")
    ctrline = lines.index("CONSTRAINTS\n")
    try:
        fixedline = lines.index("FIXED\n")
    except ValueError as err:
        fixedline = ctrline
    endline = lines.index("END\n")

    pattern = re.compile(r"  \[[^\]]+\] <(\S+)>: ")
    allvars = [
        pattern.match(line).group(1)
        for line in lines[varline:fixedline][1:]
    ]
    varlines = {
        pattern.match(line).group(1): line
        for line in lines[varline:fixedline][1:]
    }
    varorigindex = {
        v: i for i, v in enumerate(allvars)
    }
    allvars[:8]

    vardict = {v: v for v in allvars}
    vardict.update(dict(permutations))
    neworder = [vardict[v] for v in allvars]
    neworderinv = {v: i for i, v in enumerate(neworder)}

    # Write cipfile with constraints.
    with open(cipfile_withctrs, "w") as f:
        f.writelines(lines[:endline])
        f.writelines(str.replace(ctr, "t_x", "x") + "\n" for ctr in constraints)
        f.writelines(lines[endline:])

    def relabel_ctrline(ctr):
        ctrmat = re.match(r"^(\s*\[linear\] <\S+>: )((?:(?:[\+-]?)<(?:\S+)>\[B\]\s+)+)([<|=]= 1;)$", ctr)
        ctrterms = ctrmat.group(2)

        newctrterms = sorted(re.findall(r"<(\S+)>\[B\]", ctrterms), key=neworderinv.get)
        newctrtermsstr = "+".join(["<%s>[B] " % v for v in newctrterms])
        return ctrmat.group(1) + newctrtermsstr + ctrmat.group(3) + "\n"       

    # Write cipfile with constraints, relabeled.
    with open(cipfile_withrelabel, "w") as f:
        f.writelines(lines[:varline + 1])
        f.writelines(varlines[vardict[v]] for v in allvars)
        f.writelines(lines[fixedline:ctrline + 1])
        f.writelines(relabel_ctrline(ctr) for ctr in lines[ctrline + 1:endline])
        f.writelines(str.replace(ctr, "t_x", "x") + "\n" for ctr in constraints)
        f.writelines(lines[endline:])

    # Write cipfile without constraints, relabeled.
    with open(cipfile_relabelonly, "w") as f:
        f.writelines(lines[:varline + 1])
        f.writelines(varlines[vardict[v]] for v in allvars)
        f.writelines(lines[fixedline:ctrline + 1])
        f.writelines(relabel_ctrline(ctr) for ctr in lines[ctrline + 1:endline])
        # f.writelines(str.replace(ctr, "t_x", "x") + "\n" for ctr in constraints)
        f.writelines(lines[endline:])

print("Done!")